# Survival Analysis of Endometrial Cancer: Tumor Pathological Stage

Through modern statistical methods, we can determine survival risk based on a variety of factors.  In this tutorial, we will walk through a small example of something you could do with our data to understand what factors relate with survival in various different types of cancer.  In this use case, we will be looking at Endometrial Cancer

## Step 1: Import Data and Dependencies

In [ ]:
import pandas as pd
import cptac
import numpy as np
import sksurv
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import lifelines
from lifelines import KaplanMeierFitter
%matplotlib inline

In [ ]:
en = cptac.Endometrial()
clinical = en.get_clinical()
proteomics = en.get_proteomics()
follow_up = pd.read_excel('../Follow_Up_Data/UCEC_followup_9_12.xlsx')

## Step 2: Data Preparation
We will be focusing on the discovery cohort of tumors, for which we have follow-up data. We will perform some data cleaning, and then merge the tables together for analysis. While you could study a wide variety of factors related to survival, such as country of origin or number of full pregnancies, we will be focusing on tumor stage and grade.

In [ ]:
#Get just the discovery cohort
cohort_filter = follow_up['Cohort'] == "Disc"
discovery_cohort = follow_up[cohort_filter].copy()
other_cohort = follow_up[~cohort_filter]

In [ ]:
#Replace things that mess up the analysis       
to_replace = ['Not Reported/ Unknown', 'Reported/ Unknown',
              'Not Applicable', 'na', 'unknown', 'Not Performed',
              'Unknown tumor status', 'Unknown',
              'Unknown Tumor Status', 'Not specified']

for col in discovery_cohort.columns:
    discovery_cohort[col] = discovery_cohort[col].replace(to_replace, 
                                                          np.nan)

In [ ]:
#Rename column to merge on, and then merge follow-up with clinical data
discovery_cohort = discovery_cohort.rename({'Case ID': 'Patient_ID'}, axis='columns')

patient_data = pd.merge(clinical, discovery_cohort, on = 'Patient_ID')

In [ ]:
#Determine columns to focus on, and create a subset to work with
columns_to_focus_on = ['Patient_ID', 'Vital Status', 
                       'Histologic_Grade_FIGO', 'Path Diag to Last Contact(Day)', 
                       'Path Diag to Death(days)', 'Histologic_type',
                       'tumor_Stage-Pathological',
                       'Path Diag to new Tumor (days)']

focus_group = patient_data[columns_to_focus_on].copy().drop_duplicates()
focus_group = focus_group[['Vital Status',
                           'Path Diag to Last Contact(Day)',
                           'Histologic_Grade_FIGO',
                           'tumor_Stage-Pathological']].copy()

## Step 2b: Prepare data for Kaplan Meier Plotting and Survival Analysis

In [ ]:
focus_group = focus_group.replace('Living', 0)
focus_group = focus_group.replace('Deceased', 1)
focus_group = focus_group.dropna()

For Kaplan Meier plots, your data needs to be in a format similar to that shown below.  Particularly, it needs a boolean column for the 'event' you are interested in (in this case, vital status), where True denotes the event you are tracking, and False denotes an individual that never had the event of interest occur (in this case, their vital status is 'living').  It also needs a column with a numeric time frame, which we have as 'Path Diag to Last Contact(Day)'.  The other columns contain categorical data that we are testing to find meaningful connections with positive or negative event outcomes.

In [ ]:
focus_group.head()

In [ ]:
time = focus_group['Path Diag to Last Contact(Day)'].copy()
vital_status = focus_group['Vital Status'].copy()

Kaplan Meier plots show us the probability of some event occuring over a given length of time, based on some attribute.  Oftentimes, they are used to plot the probability of death for different attributes, however they could also be used in a variety of other contexts.  Below are a few examples of Kaplan Meier Plots in regards to Histologic Grade, and Tumor Stage of patients with Endometrial Cancer:

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(time, event_observed = vital_status)

In [ ]:
#groups = focus_group
kmf.plot()

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
figo_map = {"FIGO grade 1": 1, "FIGO grade 2": 2, "FIGO grade 3" : 3}
focus_group['Histologic_Grade_FIGO'] = focus_group['Histologic_Grade_FIGO'].map(figo_map)

In [ ]:
tumor_map = {"Stage I" : 1, "Stage II" : 2, "Stage III" : 3, "Stage IV" : 4}
focus_group['tumor_Stage-Pathological'] = focus_group['tumor_Stage-Pathological'].map(tumor_map)

In [ ]:
cph = CoxPHFitter()
cph.fit(focus_group, duration_col = "Path Diag to Last Contact(Day)", event_col = "Vital Status")

In [ ]:
cph.print_summary(model="untransformed variables", decimals=3)

In [ ]:
focus_group['Vital Status'].value_counts()

In [ ]:
from lifelines.statistics import proportional_hazard_test

results = proportional_hazard_test(cph, focus_group, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

In [ ]:
cph.plot()

In [ ]:
wbf = lifelines.WeibullFitter().fit(time, vital_status)

In [ ]:
wbf.plot_survival_function()

## Step 3: Separate the data for pre-processing, and to prepare for Cox Proportional Hazard Test
Cox Proportional Hazard Test is a statistical test used to interpret the significance of Kaplan Meier plots, and potential connections between attributes and an event of interest (in this case, survival).

For the test to be performed properly, our data needs to be in a specific format. It requires data about Vital Status and time to event/last contact to be in a structured array, which we have titled 'survival_array' below.  

Additionally, the attributes we are interested in studying, which are the tumor stage and histologic grade, need to be separate from this array. We separated this data into a DataFrame we call 'tumor_stage_data' below.

In [ ]:
tumor_stage_data = focus_group[["tumor_Stage-Pathological",
                                "Histologic_Grade_FIGO"]].copy()
#tumor_stage_data = pd.DataFrame(focus_group['tumor_Stage-Pathological'].copy())

survival_data = focus_group[['Vital Status',
                             "Path Diag to Last Contact(Day)"]].copy()

survival_array = np.zeros(len(survival_data),
                          dtype={"names":("Vital Status",
                                          "Path Diag to Last Contact(Day)"),
                                 "formats":("?", "<f8")})

survival_array['Vital Status'] = survival_data['Vital Status'].values
survival_array['Path Diag to Last Contact(Day)'] = survival_data['Path Diag to Last Contact(Day)'].values 

Sometimes in the conversion, merging, and manipulation of DataFrames, columns' data types get mixed up.  Sci-kit learn requires columns of type "category" or "numeric" to perform pre-processing, which will create a binary DataFrame ready for performing Cox Proportional Hazard, as well as many other predictive analyses.  Oftentimes, these data types become an "object" type, so we will change them to type "category".

In [ ]:
for col in tumor_stage_data.columns:
    tumor_stage_data[col] = tumor_stage_data[col].astype("category")

## Step 4: Perform Cox Proportional Hazard Test

In order to perform Cox Proportional Hazard Test, as well as many other Machine Learning and other statistical tests, our data needs to be pre-processed.  This is oftentimes done with Sci-kit Learn's encoder entitled "OneHotEncoder".  OneHotEncoder will create a new binary DataFrame of 0 and 1 for all the attributes in the DataFrame you give it, which will now be in the format for the test to be performed.

In [ ]:
for col in tumor_stage_data.columns:
    print(tumor_stage_data[col].value_counts(), '\n')

In [ ]:
tumor_stage_numeric = OneHotEncoder().fit_transform(tumor_stage_data)
test2 = OneHotEncoder().fit_transform(tumor_stage_data[["Histologic_Grade_FIGO", "tumor_Stage-Pathological"]])

In [ ]:
#What happened to stage I and grade 1?
tumor_stage_numeric.columns

In [ ]:
test2.columns

Here we will use our estimator of choice, Cox Proportional Hazard, to perform the test.

In [ ]:
estimator = CoxPHSurvivalAnalysis()
estimator.fit(tumor_stage_numeric, survival_array)

We can now look at our proportional hazard rations by viewing the following Series, based on the coefficients of the estimator, and the index of the pre-processed data.  These ratios help us know how influential a particular attribute is in relation to survival.  For instance, the ratio of 2.6235030 for Stage III tumors shows us that patients with a Stage III tumor had a 26.2% higher likelihood of death than if they did not have a Stage III tumor.

In [ ]:
pd.Series(estimator.coef_, index=tumor_stage_numeric.columns)

If you are interested in the accuracy of your model for predicting a positive or negative outcome based on your survival_array, you can view the estimator's "score" method.  This score is particularly relevant when testing your model against new data, but is still helpful in understanding its standalone accuracy as well.

In [ ]:
estimator.score(tumor_stage_numeric, survival_array)

Those are the basics of Survival Analysis on the cancer data we have provided in the cptac package.  This is by no means the only way this could have been done.  There are many other questions we could ask ourselves, and continue to study once we get to this point: 

How well does our model hold up for new data? What other attributes may be important for survival? What if I want to study the connection between clinical attributes and the likelihood of developing a second tumor during treatment?

With the functionality and flexibility of cptac, Sci-kit Survival and Sci-kit Learn, these answers can be explored, and a variety of other research projects could be done.  This use case is intended to be a springboard to help researchers get started in survival analysis, and leverage the cancer data we have to find important connections between clinical or molecular attributes and clinical outcomes.